# Import Libraries

In [1]:
import keras
from keras.preprocessing import image   
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import numpy as np


/Users/gordon.blackadder/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Get embeddings from images with known labels

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

model = keras.applications.VGG16(include_top=False, weights='imagenet', pooling="max")

# dimensions of our images.
img_width, img_height = 224, 224
batch_size = 13
generator = datagen.flow_from_directory(
    "images/processed",#train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False, class_mode='sparse')

label_to_id = generator.class_indices
id_to_label = {val:key for key, val in label_to_id.items()}

nb_train_samples = 67



embeddings = []
labels = []

seen_samples = 0
for image_batch, label_batch in generator:
    embs = model.predict(image_batch)
    for emb in embs:
        embeddings.append(emb)
    for lab in label_batch:
        labels.append(lab)
    seen_samples += len(label_batch)
    if seen_samples >= nb_train_samples:
        break
        
embeddings = np.array(embeddings)
labels = np.array(labels)

Found 67 images belonging to 4 classes.


In [9]:
len(embeddings)

67

In [10]:
np.save('src/embeds', embeddings)
np.save('src/labels', labels)

In [11]:
import json
with open("src/codings.json", 'w') as f:
    f.write(json.dumps(id_to_label))

# Function to import a test image

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

# Function to find most similar image from training set

In [12]:
embeddings = np.load('src/embeds.npy')
labels = np.load("src/labels.npy")

In [13]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3], dtype=int32)

In [14]:
id_to_label

{0: 'castoff', 1: 'drops', 2: 'projected', 3: 'spatters'}

In [12]:
def make_prediction(file_name):
    embeddings = np.load('src/embeds.npy')
    labels = np.load("src/labels.npy")
    test_image = path_to_tensor(file_name)
    test_embedding = model.predict(test_image)[0]
    scores = [np.dot(test_embedding, train_ex) for train_ex in embeddings]
    best_match = np.argmax(scores)
    best_label = labels[best_match]
    return id_to_label[best_label]

# Example Prediction

In [13]:
import sys
test_file_name = sys.argv[1]
make_prediction("test.jpg")

'drops'